# 🌍 GraphRAG Core: Climate Intelligence Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nunezmatias/grafoRag/blob/main/examples/Tutorial_GraphRAG.ipynb)

Welcome to the **GraphRAG Core** tutorial. This notebook demonstrates a next-generation retrieval system designed for scientific discovery.

By combining vector search with a causal knowledge graph, we can answer complex questions about climate adaptation, identifying not just *what* is happening, but *why* it matters and what ripple effects it might trigger.


## 1. Installation
Install the library directly from GitHub. This will setup the engine and download required dependencies.


In [ ]:
!pip install git+https://github.com/nunezmatias/grafoRag.git
!pip install -q -U google-genai

import os
from graphrag_core import GraphRAGEngine
print('✅ Libraries Installed & Loaded')


## 2. Initialize the Engine
Initializing the engine is simple. If the climate data is not found locally, it will be automatically downloaded from the cloud storage.


In [ ]:
engine = GraphRAGEngine()
# Output should say: "Attempting to download from Google Drive..." followed by "System Ready"


## 3. Run a Deep Research Query
We will now perform a complex search. The engine allows you to tune the depth of the investigation:

- **`top_k`** estabelece la amplitud temática.
- **`context_k`** controla la profundidad de evidencia por tema.
- **`hops`** define el razonamiento causal.


In [ ]:
# Define your research question
query = "cascading risks of extreme heat and urban floods"

# Execute the Search
results = engine.search(
    query=query, 
    top_k=3, 
    context_k=4, 
    hops=2
)

print(f"--- Research Stats ---")
print(f"Primary Sources: {results["stats"]["primary"]}")
print(f"Context Expansion: {results["stats"]["context"]}")
print(f"Causal Links:      {results["stats"]["graph"]}")


## 4. Inspect the Intelligence
Verify the quality of the retrieved data before generating the final answer.


In [ ]:
# 1. Check the Top Paper
if results['papers']:
    p = results['papers'][0]
    print(f"📄 Top Paper: {p["title"]}")
    print(f"   Snippet: {p["content"][:200]}...")

# 2. Check Discovered Causal Chains
if results['graph_links']:
    print("")
    print("🔗 Sample Causal Chains:")
    for link in results['graph_links'][:5]:
        print(f"   {link["node1"]} --[{link["relation"]}]--> {link["node2"]}")


## 5. Construct the Expert Prompt
We package the data into a rigorous prompt for the LLM using the engine's built-in template.


In [ ]:
prompt = engine.format_prompt(results, query)

print("Here is your optimized prompt (COPY THIS):")
print("--------------------------------------------------")
print(prompt)
print("--------------------------------------------------")


## 6. Generate Answer with Gemini Flash ⚡
Send the prompt to Google's Gemini Flash model for final synthesis.


In [ ]:
from google import genai
from google.colab import userdata
from IPython.display import Markdown, display

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    client = genai.Client(api_key=GOOGLE_API_KEY)
    print("✅ Gemini Client Configured")
except Exception as e:
    print("⚠️ Error: API Key not found. Please add 'GOOGLE_API_KEY' to Colab Secrets.")

print("⏳ Generating expert response with Gemini Flash...")
try:
    response = client.models.generate_content(
        model='gemini-flash-latest',
        contents=prompt
    )
    display(Markdown("### 🤖 Response:"))
    display(Markdown(response.text))
except Exception as e:
    print(f"❌ Generation Error: {e}")


## 7. Advanced: Custom Prompt Template
Access raw variables to build your own structure.


In [ ]:
my_role = "You are a Data Journalist."
my_instruction = "Summarize in 3 bullet points."
papers_text = ""
for p in results['papers']:
    papers_text += f"- {p["title"]}: {p["content"][:200]}...
"
graph_text = ""
for link in results['graph_links']:
    graph_text += f"- {link["node1"]} causes {link["node2"]}
"
custom_prompt = (f"ROLE: {my_role}
QUESTION: {query}

DATA:
{papers_text}
{graph_text}

DO: {my_instruction}")
print("--- Custom Prompt Preview ---")
print(custom_prompt[:300] + "...")


## 8. Bonus: Swap the Brain 🧠
Download and load a new domain dataset.


In [ ]:
!wget -q https://github.com/nunezmatias/grafoRag/raw/main/examples/test_brain.zip
!unzip -o -q test_brain.zip
solar_engine = GraphRAGEngine(vector_db_path='./test_brain/test_db', graph_json_path='./test_brain/test_skeleton.json')
solar_results = solar_engine.search("How does solar energy affect the grid?", top_k=2)
print(f"Found {len(solar_results["graph_links"])} causal links in the new brain.")


## 9. Advanced: Load from Google Drive ☁️
Share custom knowledge bases via Drive File IDs.


In [ ]:
MY_CUSTOM_GDRIVE_ID = "1iKcEzECN9LTMi3bIq4ocRfFJgvb1dLus"
try:
    drive_engine = GraphRAGEngine(gdrive_id=MY_CUSTOM_GDRIVE_ID)
    print("✅ Custom Brain Loaded")
except Exception as e:
    print(f"❌ Error: {e}")
